#### 분석 기법

- 나이브 베이즈 - TFid, Multi
- 감성분석 - Kosac and NB(nltk)
- 라벨링 - Kosac 사전 이용

#### 나이브 베이즈 
- TfidfVectorizer : 토큰화
- MultinomialNB : 나이브 베이즈 classify, predict

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [2]:
# 데이터 로드
article_df = pd.read_csv("tada_nb_comm_class.csv")
len(article_df)

1922

In [3]:
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922 entries, 0 to 1921
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _id         1144 non-null   object 
 1   title       1144 non-null   object 
 2   date        1144 non-null   object 
 3   youtube_id  1132 non-null   object 
 4   comment     1144 non-null   object 
 5   like_num    1144 non-null   object 
 6   class       1144 non-null   float64
dtypes: float64(1), object(6)
memory usage: 105.2+ KB


In [4]:
# 결측치 제거
article_df = article_df.dropna(axis=0, how='any')
article_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132 entries, 0 to 1921
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _id         1132 non-null   object 
 1   title       1132 non-null   object 
 2   date        1132 non-null   object 
 3   youtube_id  1132 non-null   object 
 4   comment     1132 non-null   object 
 5   like_num    1132 non-null   object 
 6   class       1132 non-null   float64
dtypes: float64(1), object(6)
memory usage: 70.8+ KB


In [5]:
# 중복제거1
test_data = list(set(article_df.comment))
len(test_data)

871

In [6]:
# 중복제거2, 감성 분석 라벨링 용
# test_data로 사용해도 무방. drop_duplicates도 써보고 싶었음.
test_data_rm_du = article_df.drop_duplicates(['comment'])
len(test_data_rm_du)

871

In [7]:
# train, test 분리. 클래스 비율 고려하지 않음.
value_ =  np.random.rand(len(article_df)) < 0.9
test = article_df[~value_]
train = article_df[value_]
len(test), len(train)
test_data = list(test.comment)

In [8]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
train.comment, train['class'], test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(923, 103, 923, 103)

In [9]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [10]:
# 학습 # 결측 발생.
model = clf.fit(X_train.values.astype("str"), y_train)

In [11]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [12]:
list(y_test[:5]), list(y_pred[:5])

([3.0, 2.0, 2.0, 3.0, 3.0], [2.0, 2.0, 2.0, 2.0, 3.0])

In [13]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        1.0       0.58      0.72      0.64        25
        2.0       0.67      0.77      0.72        44
        3.0       0.62      0.38      0.47        34

avg / total       0.63      0.63      0.62       103



#### 모델 사용하기

In [14]:
categories = {
    1: "타다",
    2: "택시",
    3: "미분류",

}

In [15]:
# 테스트용으로 사용.
# 실제 테스트는 test_data 변수 사용
contents = [
    "당연하지 타다는 불법이다",
    "공수처법때 문희상 보고 국민들 맘이 어땟는지 이제 알겠냐?",
    "타다는 나중에 딱지값 챙기려 한 사업이다..1500대면 대당 1억만 되도 1500억 이네..ㅎㅎ",
    "이재웅구속! 불법운송수익금 환수!",
    "타다영업하고싶으면제도권안으로들어와서영업해라.법도무시하고하고싶은대로하면이게나라냐?정하고싶으면다음팔은돈7천억풀어서영업해라",
    "대통령은 왜 이재웅같은 엉터리 나라시업주를 키우려 하십니까? 정말 이해가 안됩니다",
    "타다가 카택처럼 운영하면 되는거 아님? 카택도 한건데 타다는 왜? 못함?",
    "최소한 국민의 안전을 위해서라도 운송영업 면허가 있어야지. 어느나라가 무면허로 운송사업을 허가해주나.",
    "개택 해봤자 3명밖에 못 타고 타다는 15인까지 가능한데 효율면에선 타다가 나은것 같은데... 개택분들한테 묻고 싶다. 10 명 정도 단체 여행가는데 개택 이용해서 이동할지 타다이용해서 이동할지...",
    "법의테두리 안에서 장사해라~혁신은 이런데 쓰는게 아닙니다~개나소나 타다하면 법의질서가 필요할까요",
    "표얻을라고 여야가 지들 잘한거 어필하겠지 택시기사들아 우리들 뽑아줘!!!",
    "대중의 안전과 교통의 체계를 중요하게 생각해서 만들어진게 지금의 택시라고? 끝까지 자기들 부족한건 생각못하지.",
    "택시 로 취업하세요 ~~!"
    
]

In [16]:
# test_data 적용
datas = {
    "content": test_data,
    "category_code": model.predict(test_data),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,법원에서 '무죄판결을 받은 타다'. 정부는 '위법으로 만들기 위해 개정안을 바꾼다'...,1.0
1,노마적님 수고많이 하셨어요~ 여러분 우리 구독과 좋아요 많이 눌러주자구요~,3.0
2,같은 민주당이 만든법안인디....\n이씨...출마 안한다고...고추가루 뿌릴러다.....,2.0
3,헐...ㅡㅡ;;;?!!!!\n진짜 어이없네,3.0
4,저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 ...,2.0
...,...,...
101,"한국사회는 수가 많은 단체이기주의로 인하여,사회혁신이 방해 받는 일도 많아,상호 적...",1.0
102,타다기사들아! 타다고발건이 유죄로 나오면 너희들도 공범이야 범법자라고 !\n만약 ...,3.0
103,이대머리 또 구시대 신시대 택시로 구시대 랜트카로 신시대 타다 에 투자했나봐 ? ...,1.0
104,미래가어둡다.,3.0


In [17]:
round(max(model.predict_proba([contents[0]])[0]), 2)

0.9

In [18]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,법원에서 '무죄판결을 받은 타다'. 정부는 '위법으로 만들기 위해 개정안을 바꾼다'...,1.0,타다,0.73
1,노마적님 수고많이 하셨어요~ 여러분 우리 구독과 좋아요 많이 눌러주자구요~,3.0,미분류,0.50
2,같은 민주당이 만든법안인디....\n이씨...출마 안한다고...고추가루 뿌릴러다.....,2.0,택시,0.81
3,헐...ㅡㅡ;;;?!!!!\n진짜 어이없네,3.0,미분류,0.94
4,저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 ...,2.0,택시,0.98
...,...,...,...,...
101,"한국사회는 수가 많은 단체이기주의로 인하여,사회혁신이 방해 받는 일도 많아,상호 적...",1.0,타다,0.56
102,타다기사들아! 타다고발건이 유죄로 나오면 너희들도 공범이야 범법자라고 !\n만약 ...,3.0,미분류,0.58
103,이대머리 또 구시대 신시대 택시로 구시대 랜트카로 신시대 타다 에 투자했나봐 ? ...,1.0,타다,0.48
104,미래가어둡다.,3.0,미분류,0.86


In [19]:
# 모델 저장

In [20]:
pickle.dump(model, open("clf.pkl", "wb"))

In [21]:
df[df['category'] == '택시']

,content,category_code,category,proba
2,같은 민주당이 만든법안인디....\n이씨...출마 안한다고...고추가루 뿌릴러다.....,2.0,택시,0.81
4,저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 ...,2.0,택시,0.98
6,국민의 다수가 원한다고 불법을 지양할순없는겁니다. 타다는 애당초 무허가콜택시사업이었...,2.0,택시,0.97
9,"다행이다! 한국의 택시는 1억이 넘는 권리금을 안고 시작하는데, 이걸 미국의 택시와...",2.0,택시,0.97
13,면허ㆍ자격증 갖고하면 될껄 꽁짜로 쳐묵을려는사기꾼 타다 이재웅,2.0,택시,1.00
14,면허ㆍ자격증 갖고하면 될껄 꽁짜로 쳐묵을려는 사기꾼 총량제 무시 상새안 거절 ...,2.0,택시,0.99
17,희대의 사기꾼 바로 구속 가즈아,2.0,택시,0.97
18,한국정부는 택시감차정책중인데 어느4차원나라와 한국이합병을했는지요 설사 합병시에는 전...,2.0,택시,0.68
19,기업가들에게 일말의 양심도 선의도 기대해서는 안 된다. 정부가 확실하게 그들을 제어...,2.0,택시,0.49
20,타다 개편한데 친절하지 편안하지 짜증날일없지,2.0,택시,0.69


In [22]:
model.predict(test_data)

array([1., 3., 2., 3., 2., 1., 2., 3., 1., 2., 3., 3., 3., 2., 2., 3., 1.,
       2., 2., 2., 2., 2., 3., 2., 3., 2., 2., 3., 3., 2., 3., 2., 1., 1.,
       1., 2., 1., 1., 2., 1., 3., 2., 2., 3., 2., 2., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 1., 3., 3., 1., 2., 2., 2., 2., 2., 1., 2., 1., 1.,
       2., 2., 1., 2., 1., 1., 3., 1., 2., 1., 1., 2., 2., 1., 1., 2., 1.,
       2., 2., 3., 3., 2., 2., 2., 2., 3., 2., 2., 2., 3., 3., 1., 2., 1.,
       3., 1., 3., 2.])

#### 긍부정 분류

- https://github.com/seoulblanc/NaverStoreData/blob/master/ProductsAnalysis_TOP100/final_NaiveBayesClassifier.py

In [23]:
# 코드 바꾸기 귀찮아서 원 데이터를 resul로~~
result = article_df

In [24]:
from nltk.classify import NaiveBayesClassifier #Tfidf 안해도되나?
import pandas as pd
import re
# KOSAC사전에서 polarity.csv 파일을 불러온다. ngram.과 max.value 행만 사용하도록 df 내용 수정.
df_dic = pd.read_csv("polarity.csv", encoding='utf-8')
df = df_dic[df_dic['max.value'].notnull()]
df = df[['ngram','max.value']]
# 한글과 영문이 섞여있는 ngram에서, 가장 앞에 있는 한글단어만 추출하는 정규 표현식
p = r'^[가-힣]+'
# KOSAC으로 부터 긍정(POS), 부정(NEG), 중립(NEU)의 사전을 생성한다
pos_dic = []
neg_dic = []
neu_dic = []
for i, row in df.iterrows():
    if row['max.value'] ==  'POS':
        pos_dic.extend(re.findall(p, row['ngram']))
    elif row['max.value'] ==  'NEG':
        neg_dic.extend(re.findall(p, row['ngram']))
    elif row['max.value'] ==  'NEUT':
        neu_dic.extend(re.findall(p, row['ngram']))
# 중복 단어를 제거하기 위해서 set로 만들었다가 list로 변환시킨다
positive_vocab = list(set(pos_dic)) #총 1830개 단어
negative_vocab = list(set(neg_dic)) #총 1623개 단어
neutral_vocab = list(set(neu_dic)) #총 340개 단어
def word_feats(words):
    return dict((word, True) for word in words)
# 사전의 긍정, 부정, 중립단어를 navie bayes에 학습시킬 준비를 한다
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
# 트레인 데이터셋 생성 완료! naive bayes에 학습 시킨다
train_set = negative_features + positive_features + neutral_features
classifier = NaiveBayesClassifier.train(train_set)
# 예측하기 위해 데이터셋 준비(_data.csv 앞에 글자만 작성. tata를 예시로 써둠)
name = "tada"
df = result
data = df[df['comment'].notnull()]
print('총 문장수는 '+ str(len(data)) + '개 입니다')
neg = 0; pos = 0 ;neu = 0;
pos_word = ['POS'] ; neg_word = ['NEG'] ;neu_word = ['NEU'];
# for문으로 예측하고 싶은 문장을 돌려서 한 문장씩 예측 시킨다
for sentence in data['comment']:
    sentence = sentence.lower()
    words = sentence.split(',')
    for word in words:
        classResult = classifier.classify(word_feats(word))
        if classResult == 'neg':
            neg = neg + 1
            neg_word.append(word)
        elif classResult == 'pos':
            pos = pos + 1
            pos_word.append(word)
        elif classResult == 'neu' :
            neu = neu +1
            neu_word.append(word)
# 결과 출력시키기
print('긍정의 문장 수는 ' + str(pos) +'개 입니다')
print('부정의 문장 수는 ' + str(neg) +'개 입니다')
print('중립의 문장 수는 ' + str(neu) +'개 입니다')
print('긍정인 반응은 ' + str(float(pos)*100 / len(data['comment'])) + '% 입니다')
print('부정인 반응은 ' + str(float(neg)*100 /  len(data['comment'])) + '% 입니다')
print('중립인 반응은 ' + str(float(neu)*100 /  len(data['comment'])) + '% 입니다')
# 긍정, 부정, 중립의 문장을 가장 많이 나온 순으로 나열하여 csv 파일로 저장
three_type = [pos_word, neg_word, neu_word]
for one_type in three_type:
    w_count = {}
    for idx,  one_sentence in enumerate(one_type):
        if idx == 0:
            continue
        try: w_count[one_sentence] +=1
        except: w_count[one_sentence] = 1
    pd_data = pd.DataFrame(list(w_count.items()))
    pd_data.columns = ['sentence', 'count']
    pd_data = pd_data.sort_values('count', ascending=False)
    pd_data.to_csv('data_%s_%s.csv'%(name, one_type[0]), mode = 'w', index = False, encoding='utf-8', index_label= False)
print('csv 파일로(3개) 저장 완료')

총 문장수는 1132개 입니다
긍정의 문장 수는 290개 입니다
부정의 문장 수는 223개 입니다
중립의 문장 수는 823개 입니다
긍정인 반응은 25.618374558303888% 입니다
부정인 반응은 19.699646643109542% 입니다
중립인 반응은 72.70318021201413% 입니다
csv 파일로(3개) 저장 완료


##### 리스트 바로 넣기
- df 데이터만 변경 위 코드만 보면 될듯

In [25]:
from nltk.classify import NaiveBayesClassifier #Tfidf 안해도되나?
import pandas as pd
import re
# KOSAC사전에서 polarity.csv 파일을 불러온다. ngram.과 max.value 행만 사용하도록 df 내용 수정.
df_dic = pd.read_csv("subjectivity-polarity.csv", encoding='utf-8')
df = df_dic[df_dic['max.value'].notnull()]
df = df[['ngram','max.value']]
# 한글과 영문이 섞여있는 ngram에서, 가장 앞에 있는 한글단어만 추출하는 정규 표현식
p = r'^[가-힣]+'
# KOSAC으로 부터 긍정(POS), 부정(NEG), 중립(NEU)의 사전을 생성한다
pos_dic = []
neg_dic = []
neu_dic = []
for i, row in df.iterrows():
    if row['max.value'] ==  'POS':
        pos_dic.extend(re.findall(p, row['ngram']))
    elif row['max.value'] ==  'NEG':
        neg_dic.extend(re.findall(p, row['ngram']))
    elif row['max.value'] ==  'NEUT':
        neu_dic.extend(re.findall(p, row['ngram']))
# 중복 단어를 제거하기 위해서 set로 만들었다가 list로 변환시킨다
positive_vocab = list(set(pos_dic)) #총 1830개 단어
negative_vocab = list(set(neg_dic)) #총 1623개 단어
neutral_vocab = list(set(neu_dic)) #총 340개 단어
def word_feats(words):
    return dict((word, True) for word in words)
# 사전의 긍정, 부정, 중립단어를 navie bayes에 학습시킬 준비를 한다
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
# 트레인 데이터셋 생성 완료! naive bayes에 학습 시킨다
train_set = negative_features + positive_features + neutral_features
classifier = NaiveBayesClassifier.train(train_set)
# 예측하기 위해 데이터셋 준비(_data.csv 앞에 글자만 작성. tata를 예시로 써둠)
name = "tada"
df = article_df 
data = test_data
print('총 문장수는 '+ str(len(data)) + '개 입니다')
neg = 0; pos = 0 ;neu = 0;
pos_word = ['POS'] ; neg_word = ['NEG'] ;neu_word = ['NEU'];
# for문으로 예측하고 싶은 문장을 돌려서 한 문장씩 예측 시킨다
for sentence in data:
    sentence = sentence.lower()
    words = sentence.split(',')
    for word in words:
        classResult = classifier.classify(word_feats(word))
        if classResult == 'neg':
            neg = neg + 1
            neg_word.append(word)
        elif classResult == 'pos':
            pos = pos + 1
            pos_word.append(word)
        elif classResult == 'neu' :
            neu = neu +1
            neu_word.append(word)
# 결과 출력시키기
print('긍정의 문장 수는 ' + str(pos) +'개 입니다')
print('부정의 문장 수는 ' + str(neg) +'개 입니다')
print('중립의 문장 수는 ' + str(neu) +'개 입니다')
print('긍정인 반응은 ' + str(float(pos)*100 / len(data)) + '% 입니다')
print('부정인 반응은 ' + str(float(neg)*100 /  len(data)) + '% 입니다')
print('중립인 반응은 ' + str(float(neu)*100 /  len(data)) + '% 입니다')
# 긍정, 부정, 중립의 문장을 가장 많이 나온 순으로 나열하여 csv 파일로 저장
three_type = [pos_word, neg_word, neu_word]
for one_type in three_type:
    w_count = {}
    for idx,  one_sentence in enumerate(one_type):
        if idx == 0:
            continue
        try: w_count[one_sentence] +=1
        except: w_count[one_sentence] = 1
    pd_data = pd.DataFrame(list(w_count.items()))
    pd_data.columns = ['sentence', 'count']
    pd_data = pd_data.sort_values('count', ascending=False)
    pd_data.to_csv('data_%s_%s.csv'%(name, one_type[0]), mode = 'w', index = False, encoding='utf-8', index_label= False)
print('csv 파일로(3개) 저장 완료')

총 문장수는 106개 입니다
긍정의 문장 수는 25개 입니다
부정의 문장 수는 1개 입니다
중립의 문장 수는 117개 입니다
긍정인 반응은 23.58490566037736% 입니다
부정인 반응은 0.9433962264150944% 입니다
중립인 반응은 110.37735849056604% 입니다
csv 파일로(3개) 저장 완료


#### 중립제거 - 했는데 큰 의미 없었음.

======================================================================================

#### 감성분석 - kosac 이용
- https://somjang.tistory.com/entry/Keras기사-제목을-가지고-긍정-부정-중립-분류하는-모델-만들어보기

In [26]:
# NB 모델에서 분류한 kosac 긍,부,중 사전 저장
pos = positive_vocab
neg = negative_vocab
neu = neutral_vocab


In [27]:
# 각 사전별 텍스트 저장
# with 를 쓰면 close를 할 필요가 없고 라인별로 읽어줌

with open('pos.txt','w') as f:

    f.writelines(positive_vocab)
with open('neg.txt','w') as f:

    f.writelines(negative_vocab)
with open('neu.txt','w') as f:

    f.writelines(neutral_vocab)    

In [28]:
# 인덱스 to_csv에 index=False로 인덱스 안 불러오게 함.
# DataFrame index는 의미 없음
pd.DataFrame(pos, index=None).to_csv('pos.txt',index=False)
pd.DataFrame(neg, index=None).to_csv('neg.txt',index=False)
pd.DataFrame(neu, index=None).to_csv('neu.txt',index=False)

In [29]:
# 사전 생성
import codecs 
positive = [] 
negative = []
# 원코드에는 Neutral이 없었으나 추가함.
# 뻬나 안뺴나 부정을 분류 못하는거는 똑같아 보임
neutral = []
posneg = []

pos = codecs.open("pos.txt", 'rb', encoding='UTF-8')

while True: 
    line = pos.readline() 
    line = line.replace('\n', '') 
    positive.append(line) 
    posneg.append(line) 
    
    if not line: break 
pos.close()


neg = codecs.open("neg.txt", 'rb', encoding='UTF-8')

while True: 
    line = neg.readline() 
    line = line.replace('\n', '') 
    negative.append(line) 
    posneg.append(line) 
    
    if not line: break 
neg.close()

# 중립중립
neu = codecs.open("neu.txt", 'rb', encoding='UTF-8')

while True: 
    line = neu.readline() 
    line = line.replace('\n', '') 
    neutral.append(line) 
    posneg.append(line) 
    
    if not line: break 
neu.close()

In [30]:
# 결과 받을 변수 생성
# comment만 뽑아주고 후에 라벨 붙임.
# test_data 중복 제거한 데이터, test_data 사용 해도 됨.
#my_title_dic = {"comment":[], "label":label} 
my_title_dic = test_data_rm_du[["comment"]]


In [31]:
# 감성 분석 테스트 코드

titles = list((test_data_rm_du['comment']))
label = []
for i in range(len(titles)):
    clean_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', '', titles[i])
    
    for j in range(len(posneg)):
        posflag = False
        negflag = False
        if j < (len(positive)-1):
            if clean_title.find(posneg[i]) != -1:
                posflag = True
                print(j, "positive?","테스트 : ",clean_title.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", j, clean_title)
                break
                
        if j < (len(positive)-2):
            if clean_title.find(posneg[i]) != -1:
                negflag = True
                print(j, "negative?","테스트 : ",clean_title.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", j, clean_title)
                break
                
    if posflag == True:
        label.append(1)
    
    elif negflag == True:
        label.append(-1)
        
    elif negflag == False and posflag == False:
        label.append(0)
        


my_title_dic['label'] = label
my_title_dic


0 positive? 테스트 :  78 비교단어 :  0 인덱스 :  0 결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가 사납금으로 운영하는
그 사납금 때문에 택시가 승차거부하는거라며
택시는 30년 전이나 지금이나 차만 바뀌고 발전없이 똑같은데 향후 30년도 똑같게 하라는 법이네
2050년에도 승차거부 당하겠네ㅋㅋ
0 positive? 테스트 :  37 비교단어 :  는 인덱스 :  0 여상규 저 사람은 볼 때마다 소리치고있네 ㅋㅋ
지멋대로할꺼면 법사위는 왜 열었어 절차상
0 positive? 테스트 :  9 비교단어 :  넘 인덱스 :  0 먹튀 덩치키우구 넘길겁니다 재생각
0 positive? 테스트 :  0 비교단어 :  이러 인덱스 :  0 이러면 법원은 뭐가되나
0 positive? 테스트 :  19 비교단어 :  리 인덱스 :  0 정말 눈물만 난다 대한민국에서 모빌리티 서비스는 정녕 불가능한 것인가
0 positive? 테스트 :  5 비교단어 :  제 인덱스 :  0 니들은 언제 해고될지 모르는 계약직이잖아 이러고도 타다를 혁신이라고 우길레 
0 positive? 테스트 :  24 비교단어 :  용 인덱스 :  0 그동안 택시 갑질손님이 타다로 가서 택시 이용하는 갑질 개막난이들 다시 택시로 돌아오겠군 또피곤해지겠어 진상들  진상들은 택시이용하지말고 버스 지하철 타고 다니시길 부탁임
0 positive? 테스트 :  9 비교단어 :  데 인덱스 :  0 기업가가 사업하는데 법을 호소한다는게 자체가 나라가 잘못가고 있다는거임 ㅜ
더불어 민주당 문재인 대통령 찍은 내가 점점 후회스러진다ㅜㅜ
0 positive? 테스트 :  313 비교단어 :  밖 인덱스 :  0 타다 금지법은 결국 스타트업과 혁신을 막고 소비자들의 선택권을 제한하는 법이 될 겁니다
더 골 때리는 건 이 법을 민주당에서 반대하고 미래통합당이 찬성한다는 겁니다 자유시장경제 맞습니까 이게

처음부터 못하게 하던가 스타트업으로 승승장구하고 소비자들의 이용률과 만족이 높

/home/loveactualry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,comment,label
0,결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가? 사납금으로 운...,1
1,법원에서 '무죄판결을 받은 타다'. 정부는 '위법으로 만들기 위해 개정안을 바꾼다'...,0
2,렌터카콜택시가무슨혁신인가요 ㆍ 법이제한하면 당연히그만두어야하는게맞지않나요 무얼믿고 엄포야,0
3,타다 1500대인데 기사가 12000명 이면 차한대에 기사가 10명정도 라는 말인데...,0
4,노마적님 수고많이 하셨어요~ 여러분 우리 구독과 좋아요 많이 눌러주자구요~,0
...,...,...
1913,법이 없으니 또다른법으로 돌려막기 \n운수 면허 그까짓꺼 공짜로 국가에서 준것인데 ...,0
1914,그인간타다서비스중단하면내손에장을직갔다.절대서비스포기안한다.지금은동정표사기위한쇼다.두고봐라,0
1917,하나의기업이정부와기싸움해이기려고 하니 무슨빽이나힘이있나봐요 정부가 힘이없나요 기업빽...,0
1918,이재웅대표는 불법택시로 택시기사의 승객을 흠쳐갑니다.,0


In [32]:
my_title_dic.label.value_counts() # 0 중립 1 긍정 -1 부정

0    857
1     14
Name: label, dtype: int64

In [33]:
my_title_dic

,comment,label
0,결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가? 사납금으로 운...,1
1,법원에서 '무죄판결을 받은 타다'. 정부는 '위법으로 만들기 위해 개정안을 바꾼다'...,0
2,렌터카콜택시가무슨혁신인가요 ㆍ 법이제한하면 당연히그만두어야하는게맞지않나요 무얼믿고 엄포야,0
3,타다 1500대인데 기사가 12000명 이면 차한대에 기사가 10명정도 라는 말인데...,0
4,노마적님 수고많이 하셨어요~ 여러분 우리 구독과 좋아요 많이 눌러주자구요~,0
...,...,...
1913,법이 없으니 또다른법으로 돌려막기 \n운수 면허 그까짓꺼 공짜로 국가에서 준것인데 ...,0
1914,그인간타다서비스중단하면내손에장을직갔다.절대서비스포기안한다.지금은동정표사기위한쇼다.두고봐라,0
1917,하나의기업이정부와기싸움해이기려고 하니 무슨빽이나힘이있나봐요 정부가 힘이없나요 기업빽...,0
1918,이재웅대표는 불법택시로 택시기사의 승객을 흠쳐갑니다.,0
